<a href="https://colab.research.google.com/github/aditya-amlesh-jha/consistent_hashing/blob/main/consistent_hashing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
import hashlib
import random
import string

In [121]:
def generate_data(cols):
  data = []
  for _ in range(cols):
    length = random.randint(2, 12)
    random_string = ''.join(random.choices(string.ascii_letters, k=length))
    data.append(random_string)
  return data

In [122]:
class ConsistentHashing:
  def __init__(this, replicas):
    this.replicas = replicas
    this.ring = {}
    this.keys = []
    this.nodes = []

  def add_node(this, node):
    this.nodes.append(node)

    for i in range(this.replicas):
      key = this.hash_function(node.ip_address+':'+str(i))
      this.keys.append(key)
      this.ring[key] = node

    this.keys.sort()

  def remove_node(this, node):
    this.nodes.remove(node)
    for i in range(this.replicas):
      key = this.hash_function(node.ip_address+':'+str(i))
      this.keys.remove(key)
      del this.ring[key]

  def get_node(this, key):
    hash = this.hash_function(key)
    position = this.get_node_position(hash)
    return this.ring[this.keys[position]]

  def get_node_position(this, hash):

    if hash > this.keys[-1]:
      return 0

    low = 0
    high = len(this.keys)-1

    while low <= high:
      mid = low + (high-low)//2

      if this.keys[mid] == hash:
        return mid
      elif this.keys[mid] < hash:
        low = mid + 1
      else:
        high = mid - 1

    return low

  def hash_function(this, key):
    hash_object = hashlib.sha256(key.encode('utf-8'))
    hex_digest = hash_object.hexdigest()
    return int(hex_digest, 16)




In [123]:
class ServerNode:
  def __init__(this, name, ip_address):
    this.name = name
    this.ip_address = ip_address


In [124]:
ServerNodes = []

In [125]:
ServerNodes.append(ServerNode("server1", "192.168.1.1"))
ServerNodes.append(ServerNode("server2", "192.168.1.2"))
ServerNodes.append(ServerNode("server3", "192.168.1.3"))
ServerNodes.append(ServerNode("server4", "192.168.1.4"))
ServerNodes.append(ServerNode("server5", "192.168.1.5"))
ServerNodes.append(ServerNode("server6", "192.168.1.6"))

In [126]:
consistentHashing = ConsistentHashing(5)

In [127]:
nodeCount = {}

In [128]:
for node in ServerNodes:
  consistentHashing.add_node(node)

In [146]:
data = generate_data(500000)


for key in data:
  temp = consistentHashing.get_node(key).name
  if temp in nodeCount:
    nodeCount[temp] += 1
  else:
    nodeCount[temp] = 1
print()


In [147]:
for key in nodeCount.keys():
  print(key, nodeCount[key])

server6 1002020
server4 1231715
server1 1532032
server5 1603080
server2 1451364
server3 1179989
